In [1]:
from preprocessors import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pandas.read_csv("../data/nonhotel_hotel.tsv", sep="\t")
df.drop(labels=["Unnamed: 0"], axis=1, inplace=True)

Самы частотные слова, наиболее характерные для своих классов:

In [4]:
hotel_words = ["отель", "hotel", "отзыв", "beach", "resort", "дом", "гостевой", 
               "букинг", "гостиница", "сайт", "spa", "apartments", "villa", "house", 
               "residence", "grand", "hostel", "хостел", "village", "villas", "plaza", "park", "park"]
nonhotel_words = ["в", "из", "тур", "отдых", "на", "гореть", "купить", "туроператор"]

In [5]:
prep = NonHotelPreprocessor([word_count_feature, isenglish_feature, istitle_feature, many_feature, 
                             bastard_words_feature, isdigit_feature, isregion_feature, isverb_feature,
                             isadjective_feature] 
                            + [IsWordFeature(word) for word in hotel_words]
                            + [IsWordFeature(word) for word in nonhotel_words])

In [6]:
dfX = df["query"]
dfY = df["answer"]

In [7]:
dfX_train, dfX_test, dfY_train, dfY_test = train_test_split(dfX, dfY, test_size=0.3)
dfX_train.reset_index(drop=True, inplace=True)
dfX_test.reset_index(drop=True, inplace=True)

In [8]:
X_train = prep.preprocess(dfX_train)
y_train = np.array(dfY_train)
X_test = prep.preprocess(dfX_test)
y_test = np.array(dfY_test)
print(X_train.shape, y_train.shape)

(2107, 40) (2107,)


In [9]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [10]:
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.94130675526024365

Анализ ошибок:

In [11]:
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        print("right:{}, predicted:{}".format(y_test[i], y_pred[i]))
        print(dfX_test[i])
        print("predict_proba 0:{}, 1:{}".format(clf.predict_proba(X_test[i].reshape(1, -1))[0][0], 
                                                clf.predict_proba(X_test[i].reshape(1, -1))[0][1]))
        print("---------")

right:1, predicted:0
сколько стоит путевка в доминикану на 10 дней на двоих 2015 все включено 
predict_proba 0:0.77, 1:0.23
---------
right:1, predicted:0
мальдивы отзывы туристов 2016 
predict_proba 0:0.8820119047619046, 1:0.11798809523809523
---------
right:0, predicted:1
мирабель сукко
predict_proba 0:0.14973856698856694, 1:0.850261433011433
---------
right:1, predicted:0
пегас туристик самара 
predict_proba 0:0.8175431790431787, 1:0.1824568209568209
---------
right:1, predicted:0
united arab emirates 
predict_proba 0:0.9674696310978821, 1:0.032530368902117905
---------
right:0, predicted:1
находка сочи отзывы
predict_proba 0:0.49905952380952373, 1:0.5009404761904762
---------
right:0, predicted:1
отель европа золотые пески
predict_proba 0:0.32011111111111107, 1:0.6798888888888888
---------
right:0, predicted:1
улица десантная 38б геленджик россия
predict_proba 0:0.4011309523809523, 1:0.5988690476190477
---------
right:1, predicted:0
суздаль гостиницы и гостевые дома 
predict_proba 

In [12]:
clf.predict(prep.preprocess(["санкт-петербург"]))

array([1])